In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph


In [2]:
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langchain.tools import tool
from langchain_teddynote.tools import GoogleNews
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]


########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    news_tool = GoogleNews()
    return "\n".join([f'- {news["content"]}' for news in news_tool.search_by_keyword(query, k=5)])


# 도구 리스트 생성
tools = [search_keyword]

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구와 LLM 결합
llm_with_tools = llm.bind_tools(tools)


########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


# 상태 그래프 생성
graph_builder = StateGraph(State)

# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)

# 도구 노드 생성 및 추가
tool_node = ToolNode(tools=[search_keyword])

# 도구 노드 추가
graph_builder.add_node("tools", tool_node)

# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

########## 4. 엣지 추가 ##########

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")

# START > chatbot
graph_builder.add_edge(START, "chatbot")

# chatbot > END
graph_builder.add_edge("chatbot", END)

# 그래프 컴파일
graph = graph_builder.compile()

In [3]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 동기 스트림 처리(stream_mode="values")
for chunk in graph.stream(inputs, stream_mode="values"):

    # chunk 는 dictionary 형태(key: State 의 key, value: State 의 value)
    for state_key, state_value in chunk.items():
        if state_key == "messages":
            state_value[-1].pretty_print()

================================ Human Message =================================

AI 관련된 최신 뉴스를 검색해줘
================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_5J996bZEaZUeV0A9agjgUjEV)
 Call ID: call_5J996bZEaZUeV0A9agjgUjEV
  Args:
    query: AI
================================= Tool Message =================================
Name: search_keyword

- 부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟 - 지디넷코리아
- 네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것” - AI타임스
- [알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시 - 전자신문
- 엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시 - cio.com
- "엔비디아 필요 없다"…中아이플라이텍, 화웨이 칩 활용 AI 공개 - 연합뉴스
================================== Ai Message ==================================

다음은 AI와 관련된 최신 뉴스입니다:

1. **부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟** - 지디넷코리아
2. **네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것”** - AI타임스
3. **[알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시** - 전자신문
4. **엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시** - cio.com
5. **"엔비디아 필요 없다"…中아이

In [4]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 비동기 스트림 처리(stream_mode="values")
async for chunk in graph.astream(inputs, stream_mode="values"):
    # chunk 는 dictionary 형태(key: State 의 key, value: State 의 value)
    for state_key, state_value in chunk.items():
        if state_key == "messages":
            state_value[-1].pretty_print()

================================ Human Message =================================

AI 관련된 최신 뉴스를 검색해줘
================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_lEhAHCHsMk8YnLSmjNjqzYyr)
 Call ID: call_lEhAHCHsMk8YnLSmjNjqzYyr
  Args:
    query: AI
================================= Tool Message =================================
Name: search_keyword

- 부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟 - 지디넷코리아
- 네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것” - AI타임스
- [알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시 - 전자신문
- 엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시 - cio.com
- "엔비디아 필요 없다"…中아이플라이텍, 화웨이 칩 활용 AI 공개 - 연합뉴스
================================== Ai Message ==================================

최신 AI 관련 뉴스는 다음과 같습니다:

1. **부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟** - 지디넷코리아
2. **네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것”** - AI타임스
3. **[알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시** - 전자신문
4. **엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시** - cio.com
5. **"엔비디아 필요 없다"…中아

In [5]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

final_result = None

# 비동기 스트림 처리(stream_mode="values")
async for chunk in graph.astream(inputs, stream_mode="values"):
    final_result = chunk

# 최종 결과 출력
print(final_result["messages"][-1].content)

다음은 AI와 관련된 최신 뉴스입니다:

1. **부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟** - 지디넷코리아
2. **네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것”** - AI타임스
3. **[알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시** - 전자신문
4. **엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시** - cio.com
5. **"엔비디아 필요 없다"…中아이플라이텍, 화웨이 칩 활용 AI 공개** - 연합뉴스

더 궁금한 내용이 있으면 말씀해 주세요!


In [6]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 동기 스트림 처리(stream_mode="updates")
for chunk in graph.stream(inputs, stream_mode="updates"):
    # chunk 는 dictionary 형태(key: 노드, value: 노드의 상태 값)
    for node, value in chunk.items():
        if node:
            print(f"\n[Node: {node}]\n")
        if "messages" in value:
            value["messages"][-1].pretty_print()


[Node: chatbot]

================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_KYseL02rYpYcnKvR4dg7OgMe)
 Call ID: call_KYseL02rYpYcnKvR4dg7OgMe
  Args:
    query: AI

[Node: tools]

================================= Tool Message =================================
Name: search_keyword

- 부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟 - 지디넷코리아
- 네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것” - AI타임스
- [알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시 - 전자신문
- 엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시 - cio.com
- "엔비디아 필요 없다"…中아이플라이텍, 화웨이 칩 활용 AI 공개 - 연합뉴스

[Node: chatbot]

================================== Ai Message ==================================

다음은 AI 관련된 최신 뉴스입니다:

1. **부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟** - 지디넷코리아
2. **네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것”** - AI타임스
3. **[알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시** - 전자신문
4. **엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시** - cio.com
5. **"엔비디아 필요 없다"…中아이플라이텍, 화웨이 칩 활용 AI 공개** - 연합뉴스

더 궁금한 점이 있으시면 말씀해 주

In [7]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 동기 스트림 처리(stream_mode="messages")
# chunk_msg: 실시간 출력 메시지, metadata: 노드 정보
for chunk_msg, metadata  in graph.stream(inputs, stream_mode="messages"):

    # chatbot 노드에서 출력된 메시지만 출력
    if metadata["langgraph_node"] == "chatbot":
        if chunk_msg.content:
            print(chunk_msg.content, end="", flush=True)

    else:
        print(chunk_msg.content)
        print(f"\n\nmetadata: \n{metadata}\n\n")

- 부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟 - 지디넷코리아
- 네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것” - AI타임스
- [알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시 - 전자신문
- 엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시 - cio.com
- "엔비디아 필요 없다"…中아이플라이텍, 화웨이 칩 활용 AI 공개 - 연합뉴스


metadata: 
{'langgraph_step': 2, 'langgraph_node': 'tools', 'langgraph_triggers': ('branch:to:tools',), 'langgraph_path': ('__pregel_pull', 'tools'), 'langgraph_checkpoint_ns': 'tools:25de8c4b-7bc5-de5a-a17e-84337b53a03c'}


다음은 AI 관련 최신 뉴스입니다:

1. **부정행위 천재 대학생 둘, AI 회사 차려 75억원 잭팟** - 출처: 지디넷코리아
2. **네이버클라우드, 경량 모델 3종 오픈 소스 공개** - “실제 서비스 바꾸는 AI 선보일 것” - 출처: AI타임스
3. **IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시** - 출처: 전자신문
4. **엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시** - 출처: cio.com
5. **"엔비디아 필요 없다"... 中아이플라이텍, 화웨이 칩 활용 AI 공개** - 출처: 연합뉴스

더 자세한 내용을 원하시면 각 출처에서 기사를 확인해 보세요!

In [8]:
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langchain.tools import tool
from langchain_teddynote.tools import GoogleNews
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]


########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    news_tool = GoogleNews()
    return "\n".join([f'- {news["content"]}' for news in news_tool.search_by_keyword(query, k=5)])


# 도구 리스트 생성
tools = [search_keyword]

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구와 LLM 결합 (tags 추가)
llm_with_tools = llm.bind_tools(tools).with_config(tags=["WANT_TO_STREAM"])

########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


# 상태 그래프 생성
graph_builder = StateGraph(State)

# 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)

# 도구 노드 생성 및 추가
tool_node = ToolNode(tools=[search_keyword])

# 도구 노드 추가
graph_builder.add_node("tools", tool_node)

# 조건부 엣지
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

########## 4. 엣지 추가 ##########

# tools > chatbot
graph_builder.add_edge("tools", "chatbot")

# START > chatbot
graph_builder.add_edge(START, "chatbot")

# chatbot > END
graph_builder.add_edge("chatbot", END)

# 그래프 컴파일
graph = graph_builder.compile()

In [9]:
from langchain_core.messages import HumanMessage

# 사용자의 메시지를 딕셔너리 형태로 입력 데이터 구성
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# stream_mode="messages" 를 통한 스트리밍 처리
for chunk_msg, metadata in graph.stream(inputs, stream_mode="messages"):
    # HumanMessage 가 아닌 최종 노드의 유효한 컨텐츠만 출력 처리
    if (
        chunk_msg.content
        and not isinstance(chunk_msg, HumanMessage)
        and metadata["langgraph_node"] == "chatbot"
    ):
        print(chunk_msg.content, end="", flush=True)

최근 AI 관련 뉴스는 다음과 같습니다:

1. **부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟** - 지디넷코리아
2. **네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것”** - AI타임스
3. **[알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시** - 전자신문
4. **엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시** - cio.com
5. **"엔비디아 필요 없다"…中아이플라이텍, 화웨이 칩 활용 AI 공개** - 연합뉴스

더 궁금한 내용이 있으면 말씀해 주세요!

In [10]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 비동기 이벤트 스트림 처리(astream_events)
async for event in graph.astream_events(inputs, version="v2"):
    # 이벤트 종류와 태그 정보 추출
    kind = event["event"]
    tags = event.get("tags", [])

    # 채팅 모델 스트림 이벤트 및 최종 노드 태그 필터링
    if kind == "on_chat_model_stream" and "WANT_TO_STREAM" in tags:
        # 이벤트 데이터 추출
        data = event["data"]

        # 출력 메시지
        if data["chunk"].content:
            print(data["chunk"].content, end="", flush=True)

다음은 AI 관련 최신 뉴스입니다:

1. **부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟** - 지디넷코리아
2. **네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것”** - AI타임스
3. **[알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시** - 전자신문
4. **엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시** - cio.com
5. **"엔비디아 필요 없다"...中아이플라이텍, 화웨이 칩 활용 AI 공개** - 연합뉴스

각 기사의 내용을 더 자세히 알고 싶으시면 말씀해 주세요!

In [11]:
from langchain_core.messages import AIMessageChunk, HumanMessage

# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 첫 번째 메시지 처리 여부 플래그 설정
first = True

# 비동기 스트림 처리를 통한 메시지 및 메타데이터 순차 처리
for msg, metadata in graph.stream(inputs, stream_mode="messages"):
    # 사용자 메시지가 아닌 경우의 컨텐츠 출력 처리
    if msg.content and not isinstance(msg, HumanMessage):
        print(msg.content, end="", flush=True)

    # AI 메시지 청크 처리 및 누적
    if isinstance(msg, AIMessageChunk):
        if first:
            gathered = msg
            first = False
        else:
            gathered = gathered + msg

        # 도구 호출 청크 존재 시 누적된 도구 호출 정보 출력
        if msg.tool_call_chunks:
            print(gathered.tool_calls[0]["args"])

{}
{}
{}
{'query': ''}
{'query': 'AI'}
{'query': 'AI'}
- 부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟 - 지디넷코리아
- 네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것” - AI타임스
- [알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시 - 전자신문
- 엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시 - cio.com
- "엔비디아 필요 없다"…中아이플라이텍, 화웨이 칩 활용 AI 공개 - 연합뉴스다음은 AI 관련 최신 뉴스입니다:

1. **부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟** - 지디넷코리아
2. **네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것”** - AI타임스
3. **[알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시** - 전자신문
4. **엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시** - cio.com
5. **"엔비디아 필요 없다"…중국 아이플라이텍, 화웨이 칩 활용 AI 공개** - 연합뉴스

더 궁금한 점이 있다면 말씀해 주세요!

In [12]:
from typing import Annotated, List, Dict
from typing_extensions import TypedDict
from langchain.tools import tool
from langchain_teddynote.tools import GoogleNews
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


########## 1. 상태 정의 ##########
# 상태 정의
class State(TypedDict):
    # 메시지 목록 주석 추가
    messages: Annotated[list, add_messages]


########## 2. 도구 정의 및 바인딩 ##########
# 도구 초기화
# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    news_tool = GoogleNews()
    return "\n".join([f'- {news["content"]}' for news in news_tool.search_by_keyword(query, k=5)])


# 도구 리스트 생성
tools = [search_keyword]

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구와 LLM 결합 (tags 추가)
llm_with_tools = llm.bind_tools(tools).with_config(tags=["WANT_TO_STREAM"])

########## 3. 노드 추가 ##########
# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# SNS 포스트 생성 함수 정의
def create_sns_post(state: State):
    # SNS 포스트 생성을 위한 프롬프트
    sns_prompt = """
    이전 대화 내용을 바탕으로 SNS 게시글 형식으로 변환해주세요.
    다음 형식을 따라주세요:
    - 해시태그 포함
    - 이모지 사용
    - 간결하고 흥미로운 문체 사용
    - 200자 이내로 작성
    """
    messages = state["messages"] + [("human", sns_prompt)]
    sns_llm = ChatOpenAI(model="gpt-4o-mini").with_config(tags=["WANT_TO_STREAM2"])
    return {"messages": [sns_llm.invoke(messages)]}

# 서브그래프 생성
def create_subgraph():
    # 서브그래프용 상태 그래프 생성
    subgraph = StateGraph(State)

    # 챗봇 노드 추가
    subgraph.add_node("chatbot", chatbot)

    # 도구 노드 생성 및 추가
    tool_node = ToolNode(tools=[search_keyword])
    subgraph.add_node("tools", tool_node)

    # 조건부 엣지 추가
    subgraph.add_conditional_edges(
        "chatbot",
        tools_condition,
    )

    # tools > chatbot
    subgraph.add_edge("tools", "chatbot")

    # START > chatbot
    subgraph.add_edge(START, "chatbot")

    # chatbot > END
    subgraph.add_edge("chatbot", END)

    return subgraph.compile()

# 메인 그래프 생성
graph_builder = StateGraph(State)

# 서브그래프 추가
subgraph = create_subgraph()
graph_builder.add_node("news_subgraph", subgraph)

# SNS 포스트 생성 노드 추가
graph_builder.add_node("sns_post", create_sns_post)

# START > news_subgraph
graph_builder.add_edge(START, "news_subgraph")

# news_subgraph > sns_post
graph_builder.add_edge("news_subgraph", "sns_post")

# sns_post > END
graph_builder.add_edge("sns_post", END)

# 그래프 컴파일
graph = graph_builder.compile()

In [13]:
# 질문 입력
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 노드 업데이트 정보 순차적 처리 및 출력
for chunk in graph.stream(inputs, stream_mode="updates"):
    # node_name: 현재 처리 중인 노드명, node_chunk: 해당 노드의 청크 데이터
    for node_name, node_chunk in chunk.items():
        # 현재 처리 중인 노드 구분선 출력
        print(f"\n========= Update from node {node_name} =========\n")
        # 해당 노드의 업데이트된 데이터 출력
        if "messages" in node_chunk:
            node_chunk["messages"][-1].pretty_print()
        else:
            print(node_chunk)



========= Update from node news_subgraph =========

================================== Ai Message ==================================

최신 AI 관련 뉴스는 다음과 같습니다:

1. **부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟** - 지디넷코리아
2. **네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것”** - AI타임스
3. **[알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시** - 전자신문
4. **엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시** - cio.com
5. **"엔비디아 필요 없다"…中아이플라이텍, 화웨이 칩 활용 AI 공개** - 연합뉴스

더 궁금한 내용이 있으면 말씀해 주세요!

========= Update from node sns_post =========

================================== Ai Message ==================================

🚀 최신 AI 소식! 🎉 부정행위 천재 대학생들이 AI 회사를 차려 무려 75억원의 잭팟을 터뜨렸다고! 🤯 네이버클라우드도 경량 AI 모델 3종을 오픈 소스 공개했어요!💡💻 엔비디아는 AI 에이전트 엔진을 출시하고, 중국 아이플라이텍은 화웨이 칩을 활용한 AI를 선보였네요! 🇨🇳🔥 

#AI #딥러닝 #테크뉴스 #스타트업 #혁신 #네이버클라우드


In [14]:
# 사용자의 메시지를 딕셔너리 형태로 입력 데이터 구성
inputs = {"messages": [("human", "AI 관련된 최신 뉴스를 검색해줘")]}

# 네임스페이스 문자열을 보기 좋은 형식으로 변환하는 포맷팅 함수
def format_namespace(namespace):
    return (
        namespace[-1].split(":")[0]
        if len(namespace) > 0
        else "parent graph"
    )

# subgraphs=True 를 통해 서브그래프의 출력도 포함(namespace, chunk) 형태로 출력됩니다.
for namespace, chunk in graph.stream(inputs, stream_mode="updates", subgraphs=True):
    # node_name: 현재 처리 중인 노드명, node_chunk: 해당 노드의 청크 데이터
    for node_name, node_chunk in chunk.items():
        print(f"\n========= Update from node [{node_name}] in [{format_namespace(namespace)}] =========\n")

        # 노드의 청크 데이터 출력
        if "messages" in node_chunk:
            node_chunk["messages"][-1].pretty_print()
        else:
            print(node_chunk)


========= Update from node [chatbot] in [news_subgraph] =========

================================== Ai Message ==================================
Tool Calls:
  search_keyword (call_6QnUMUn7OUKLyVNPgA7x7k0u)
 Call ID: call_6QnUMUn7OUKLyVNPgA7x7k0u
  Args:
    query: AI

========= Update from node [tools] in [news_subgraph] =========

================================= Tool Message =================================
Name: search_keyword

- 부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟 - 지디넷코리아
- 네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것” - AI타임스
- [알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시 - 전자신문
- 엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시 - cio.com
- "엔비디아 필요 없다"…中아이플라이텍, 화웨이 칩 활용 AI 공개 - 연합뉴스

========= Update from node [chatbot] in [news_subgraph] =========

================================== Ai Message ==================================

다음은 최근 AI 관련 뉴스입니다:

1. **부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟** - 지디넷코리아
2. **네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것”** - AI타임스
3. **[알림] IT메가비전 2025, 관세전쟁

In [15]:
# 네임스페이스 정보를 파싱하는 함수
def parse_namespace_info(info: tuple) -> tuple[str, str]:
    if len(info) > 1:
        namespace, node_name = info
        return node_name.split(":")[0], namespace.split(":")[0]
    return info[0].split(":")[0], "parent graph"

kind = None

async for event in graph.astream_events(inputs, version="v2", subgraphs=True):
    kind = event["event"]

    # 이벤트 종류와 태그 정보 추출
    if kind == "on_chat_model_start":
        print(f"\n========= on_chat_model_start =========\n")

    # 채팅 모델 스트림 이벤트 및 최종 노드 태그 필터링
    elif kind == "on_chat_model_stream":
        # 이벤트 데이터 추출
        data = event["data"]

        # 토큰 단위의 스트리밍 출력
        if data["chunk"].content:
            print(data["chunk"].content, end="", flush=True)

    elif kind == "on_tool_start":
        print(f"\n========= tool_start =========\n")
        data = event["data"]
        if "input" in data:
            tool_msg = data["input"]
            print(tool_msg)

    elif kind == "on_tool_end":
        print(f"\n========= tool_end =========\n")
        data = event["data"]
        if "output" in data:
            tool_msg = data["output"]
            print(tool_msg.content)


========= on_chat_model_start =========


========= tool_start =========

{'query': 'AI'}

========= tool_end =========

- 부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟 - 지디넷코리아
- 네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것” - AI타임스
- [알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시 - 전자신문
- 엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시 - cio.com
- "엔비디아 필요 없다"…中아이플라이텍, 화웨이 칩 활용 AI 공개 - 연합뉴스

========= on_chat_model_start =========

다음은 AI 관련 최신 뉴스입니다:

1. **부정행위 천재 대학생 둘...AI 회사 차려 75억원 잭팟** - 지디넷코리아
2. **네이버클라우드, 경량 모델 3종 오픈 소스 공개...“실제 서비스 바꾸는 AI 선보일 것”** - AI타임스
3. **[알림] IT메가비전 2025, 관세전쟁 대응과 AI 육성전략 제시** - 전자신문
4. **엔비디아, AI 에이전트 엔진 ‘네모 마이크로서비스’ 공식 출시** - cio.com
5. **"엔비디아 필요 없다"…中아이플라이텍, 화웨이 칩 활용 AI 공개** - 연합뉴스

이들 뉴스는 최근 AI 분야에서의 다양한 발전과 사건들을 다루고 있습니다.
========= on_chat_model_start =========

🚀 AI가 뜨고 있어요! 최근 부정행위 천재 대학생들이 AI 회사로 75억 대박! 💰 네이버클라우드, 서비스 혁신을 위한 경량 AI 모델 공개! 🔥 엔비디아는 새로운 AI 에이전트 출시! 📈 중국은 화웨이 칩으로 AI 기술을 강화하고 있답니다! #AI #기술혁신 #네이버클라우드 #엔비디아 #화웨이